# LangChain Agents

In [5]:
pip install -r ./files/requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


## Python dotenv

In [6]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

## Python REPL

In [7]:
pip install -q langchain_experimental

Note: you may need to restart the kernel to use updated packages.


In [8]:
from langchain_experimental.utilities import PythonREPL
python_repl = PythonREPL()
python_repl.run('print([n for n in range(1, 100) if n % 13 ==0])')

Python REPL can execute arbitrary code. Use with caution.


'[13, 26, 39, 52, 65, 78, 91]\n'

In [9]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0)
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)

agent_executor.invoke('Calculate the square root of the factorial of 12 and display it with 2 decimal points')



> Entering new AgentExecutor chain...
To solve this, I will first calculate the factorial of 12 using the math module's factorial function. Then, I will calculate the square root of that result using the math module's sqrt function. Finally, I will format the result to display it with 2 decimal points.
Action: Python_REPL
Action Input: import math
print(f"{math.sqrt(math.factorial(12)):.2f}")
Observation: 21886.11

Thought:I now know the final answer
Final Answer: 21886.11

> Finished chain.


{'input': 'Calculate the square root of the factorial of 12 and display it with 2 decimal points',
 'output': '21886.11'}

In [12]:
response = agent_executor.invoke('What is the answer to 5.1 ** 7.3?')



> Entering new AgentExecutor chain...
I will calculate the power of 5.1 raised to 7.3 using Python.
Action: Python_REPL
Action Input: print(5.1 ** 7.3)
Observation: 146306.05007233328

Thought:I now know the final answer
Final Answer: 146306.05007233328

> Finished chain.


In [13]:
response

{'input': 'What is the answer to 5.1 ** 7.3?', 'output': '146306.05007233328'}

In [14]:
print(response['input'])

What is the answer to 5.1 ** 7.3?


In [15]:
print(response['output'])

146306.05007233328


## LangChain Tools: DuckDuckGo and Wikipedia

In [16]:
pip install -q duckduckgo-search

Note: you may need to restart the kernel to use updated packages.


In [21]:
# optional, filter workings (recommended in production)
import warnings
warnings.filterwarnings('ignore', module='langchain')

In [22]:
from ddgs import DDGS

def search_duckduckgo(query, max_results=3):
    with DDGS() as ddgs:
        results = ddgs.text(query)
        return [res["body"] for res in results[:max_results]]

output = search_duckduckgo('Where was Freddie Mercury born?')
print(output)

['6 days ago - Mercury defied the conventions of a rock frontman with his theatrical style, influencing the artistic direction of Queen. Born in 1946 in Zanzibar to Parsi-Indian parents, Mercury attended British boarding schools in India from the age of eight and returned to Zanzibar after secondary school.', '1 day ago - Freddie Mercury was born Farrokh Bulsara on 5 September 1946 in Stone Town in the British protectorate of the Zanzibar (now part of Tanzania). His parents were Jer and Bomi Bulsara, who were both Zoroastrians, from India.His father worked as a cashier in the British Colonial Office, a branch ...', 'July 10, 2025 - Freddie Mercury (born September 5, 1946, Stone Town, Zanzibar [now in Tanzania]—died November 24, 1991, Kensington, London, England) was a British rock singer and songwriter whose flamboyant showmanship and powerfully agile vocals, most famously for the band Queen, made him one of rock’s ...']


In [23]:
from langchain_core.tools import tool
from ddgs import DDGS

@tool
def duckduckgo_search(query: str) -> str:
    """Search DuckDuckGo for recent or relevant info."""
    with DDGS() as ddgs:
        results = ddgs.text(query)
        return results[0]["body"] if results else "No results found"

output = duckduckgo_search.invoke("Freddie Mercury and Queen")
print(output)

Freddie Mercury (born Farrokh Bulsara; 5 September 1946 – 24 November 1991) was a British singer and songwriter who achieved global fame as the lead vocalist and pianist of the rock band Queen .


In [25]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain.tools import DuckDuckGoSearchResults

wrapper = DuckDuckGoSearchAPIWrapper(region='de-de', max_results=3, safesearch='moderate')
search = DuckDuckGoSearchResults(api_wrapper=wrapper, source='news')
output = search.run('Berlin')
print(output)

/opt/anaconda3/lib/python3.12/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


snippet: 15. Mai 2025 · Siemens Energy will ein Hochhaus in Berlin bauen, am Siemens Energy Standort Huttenstraße. Es soll ein repräsentatives Gebäude unter anderem für die eigene …, title: Berlin | Siemens Energy Hochhaus am Standort Huttenstraße | In …, link: https://www.skyscrapercity.com/threads/berlin-siemens-energy-hochhaus-am-standort-huttenstraße-in-planung.2436153/, snippet: 31. Mai 2011 · The Berlin City Palace or Berlin Palace (Berliner Stadtschloss aka Berliner Schloss in German) is (partly) getting reconstructed. It's scheduled to be finished and opened as the …, title: BERLIN | City Palace Reconstruction ... - SkyscraperCity Forum, link: https://www.skyscrapercity.com/threads/berlin-city-palace-reconstruction-stadtschloss-humboldt-forum-u-c.1390928/, snippet: 25. März 2015 · Berlin | Covivio Tower | 130m | In Bau alex alexanderplatz baustelle blackstone investor kneipe kollhoff neubau parkinn primark projekt sockelbau stadtkrone strandbar 2 …, title: Berlin | Covivio Towe

In [27]:
search.name

'duckduckgo_results_json'

In [28]:
search.description

'A wrapper around Duck Duck Go Search. Useful for when you need to answer questions about current events. Input should be a search query.'

In [26]:
import re

pattern = r'snippet:\s*(.*?),\s*title:\s*(.*?),\s*link:\s*(https?://[^\s,]+)'
matches = re.findall(pattern, output, re.DOTALL)

for snippet, title, link in matches:
    print(f'Snippet: {snippet}\nTitle: {title}\nLink: {link}\n')
    print('-' * 50)

Snippet: 15. Mai 2025 · Siemens Energy will ein Hochhaus in Berlin bauen, am Siemens Energy Standort Huttenstraße. Es soll ein repräsentatives Gebäude unter anderem für die eigene …
Title: Berlin | Siemens Energy Hochhaus am Standort Huttenstraße | In …
Link: https://www.skyscrapercity.com/threads/berlin-siemens-energy-hochhaus-am-standort-huttenstraße-in-planung.2436153/

--------------------------------------------------
Snippet: 31. Mai 2011 · The Berlin City Palace or Berlin Palace (Berliner Stadtschloss aka Berliner Schloss in German) is (partly) getting reconstructed. It's scheduled to be finished and opened as the …
Title: BERLIN | City Palace Reconstruction ... - SkyscraperCity Forum
Link: https://www.skyscrapercity.com/threads/berlin-city-palace-reconstruction-stadtschloss-humboldt-forum-u-c.1390928/

--------------------------------------------------
Snippet: 25. März 2015 · Berlin | Covivio Tower | 130m | In Bau alex alexanderplatz baustelle blackstone investor kneipe kollho

### Wikipedia

In [29]:
pip install -q wikipedia

/opt/anaconda3/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=31206) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [30]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=5000)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.invoke({'query': 'llamaindex'})

'Page: Vector database\nSummary: A vector database, vector store or vector search engine is a database that uses the vector space model to store vectors (fixed-length lists of numbers) along with other data items. Vector databases typically implement one or more approximate nearest neighbor algorithms, so that one can search the database with a query vector to retrieve the closest matching database records.\nVectors are mathematical representations of data in a high-dimensional space. In this space, each dimension corresponds to a feature of the data, with the number of dimensions ranging from a few hundred to tens of thousands, depending on the complexity of the data being represented. A vector\'s position in this space represents its characteristics. Words, phrases, or entire documents, as well as images, audio, and other types of data, can all be vectorized.\nThese feature vectors may be computed from the raw data using machine learning methods such as feature extraction algorithms,

In [31]:
wiki.invoke('Google Gemini')

"Page: Gemini (chatbot)\nSummary: Gemini is a generative artificial intelligence chatbot developed by Google. Based on the large language model (LLM) of the same name, it was launched in February 2024. Its predecessor, Bard, was launched in March 2023 in response to the rise of OpenAI's ChatGPT and was based on the LaMDA and PaLM LLMs.\n\n"

In [32]:
print(output)

snippet: 15. Mai 2025 · Siemens Energy will ein Hochhaus in Berlin bauen, am Siemens Energy Standort Huttenstraße. Es soll ein repräsentatives Gebäude unter anderem für die eigene …, title: Berlin | Siemens Energy Hochhaus am Standort Huttenstraße | In …, link: https://www.skyscrapercity.com/threads/berlin-siemens-energy-hochhaus-am-standort-huttenstraße-in-planung.2436153/, snippet: 31. Mai 2011 · The Berlin City Palace or Berlin Palace (Berliner Stadtschloss aka Berliner Schloss in German) is (partly) getting reconstructed. It's scheduled to be finished and opened as the …, title: BERLIN | City Palace Reconstruction ... - SkyscraperCity Forum, link: https://www.skyscrapercity.com/threads/berlin-city-palace-reconstruction-stadtschloss-humboldt-forum-u-c.1390928/, snippet: 25. März 2015 · Berlin | Covivio Tower | 130m | In Bau alex alexanderplatz baustelle blackstone investor kneipe kollhoff neubau parkinn primark projekt sockelbau stadtkrone strandbar 2 …, title: Berlin | Covivio Towe

## Creating a ReAct Agent

In [33]:
pip install langchainhub -q

/opt/anaconda3/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=31206) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [34]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-4-turbo-preview', temperature=0)

template = '''
Answer the following questions as best you can.
Questions: {q}
'''
prompt_template = PromptTemplate.from_template(template)

#Fetches a pre-built ReAct prompt from the LangChain Hub
prompt = hub.pull('hwchase17/react')

print(type(prompt))
print(prompt.input_variables)
print(prompt.template)

/opt/anaconda3/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


<class 'langchain_core.prompts.prompt.PromptTemplate'>
['agent_scratchpad', 'input', 'tool_names', 'tools']
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [35]:
# 1. Python REPL Tool (for executing Python code)
python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name='Python REPL',
    func=python_repl.run,
    description='Useful when you need to use Python to answer a question. You should input Python code.'
)

# 2. Wikipedia Tool (for searching Wikipedia)
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name='Wikipedia',
    func=wikipedia.run,
    description='Useful for when you need to look up a topic, country, or person on Wikipedia.'
)

# 3. DuckDuckGo Search Tool (for general web searches)
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func=search.run,
    description='Useful for when you need to perform an internet search to find information that another tool can\'t provide.'
)

tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)

In [36]:
question = 'Generate the first 20 numbers in the Fibonacci series.'
output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
})



> Entering new AgentExecutor chain...
To generate the first 20 numbers in the Fibonacci series, I can write a simple Python program that iterates 20 times, each time calculating the next number in the series by summing up the two preceding numbers.

Action: Python REPL

Action Input:
```python
def fibonacci(n):
    fib_series = [0, 1]
    for i in range(2, n):
        next_fib = fib_series[-1] + fib_series[-2]
        fib_series.append(next_fib)
    return fib_series

fibonacci(20)
```
I now know the final answer

Final Answer: The first 20 numbers in the Fibonacci series are: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181.

> Finished chain.


In [37]:
print(output['input'])


Answer the following questions as best you can.
Questions: Generate the first 20 numbers in the Fibonacci series.



In [38]:
print(output['output'])

The first 20 numbers in the Fibonacci series are: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181.


In [39]:
question2 = 'Who is the current prime minister of the UK?'
output2 = agent_executor.invoke({
    'input': prompt_template.format(q=question2)
})

print(output2['input'])
print(output2['output'])



> Entering new AgentExecutor chain...
To find out the current Prime Minister of the UK, I can use the Wikipedia tool to get the most up-to-date information.

Action: Wikipedia

Action Input: Current Prime Minister of the UK
Page: List of prime ministers of the United Kingdom
Summary: The prime minister of the United Kingdom is the principal minister of the crown of His Majesty's Government, and the head of the British Cabinet. 
There is no specific date for when the office of prime minister first appeared, as the role was not created but rather evolved over time through a merger of duties. The term was regularly, if informally, used by Robert Walpole by the 1730s. It was used in the House of Commons as early as 1805, and it was certainly in parliamentary use by the 1880s, although did not become the official title until 1905, when Henry Campbell-Bannerman was prime minister.  
Historians generally consider Robert Walpole, who led the government of the Kingdom of Great Britain for ove

In [40]:
question3 = 'Tell me about Napoleon Bonaparte early life'
output3 = agent_executor.invoke({
    'input': prompt_template.format(q=question3)
})

print(output3['input'])
print(output3['output'])



> Entering new AgentExecutor chain...
To answer this question, I can use the Wikipedia tool to look up information about Napoleon Bonaparte's early life.

Action: Wikipedia

Action Input: Napoleon Bonaparte
Page: Napoleon
Summary: Napoleon Bonaparte (born Napoleone di Buonaparte; 15 August 1769 – 5 May 1821), later known by his regnal name Napoleon I, was a French general and statesman who rose to prominence during the French Revolution and led a series of military campaigns across Europe during the French Revolutionary and Napoleonic Wars from 1796 to 1815. He led the French Republic as First Consul from 1799 to 1804, then ruled the French Empire as Emperor of the French from 1804 to 1814, and briefly again in 1815. He was King of Italy from 1805 to 1814 and Protector of the Confederation of the Rhine from 1806 to 1813.
Born on the island of Corsica to a family of Italian origin, Napoleon moved to mainland France in 1779 and was commissioned as an officer in the French Royal Army in